In [1]:
from datetime import datetime
from genericpath import sameopenfile
from re import sub
from sqlalchemy import create_engine
import psycopg2
import json

from pandas.io.json._normalize import nested_to_record
import pprint as pp
import pandas as pd
from typing import List

from typing import TypeVar
PandasDataFrame = TypeVar('pandas.core.frame.DataFrame')
pd.set_option('display.max_columns', None)

In [9]:


bills = [{'contentAccountRef': {'contentProviderId': 2, 'contentProviderName': 'Intuit Mint Bills', 'contentLoginRef': 'c52b0209-04d5-3922-b6cb-8cbb2bae22f3', 'contentAccountRef': '79fb33687147ae2cb28aa07af3739811', 'connectedAccounts': [{'contentProviderId': 2, 
'contentProviderName': 'Intuit Mint Bills', 'contentLoginRef': 'c52b0209-04d5-3922-b6cb-8cbb2bae22f3', 'contentAccountRef': '79fb33687147ae2cb28aa07af3739811'}]}, 'providerRef': {'providerId': 'da579b9a-a53a-4a0d-91e3-8cf9c75fca28', 'providerName': 'xfinity', 'contentProviderRef': [{'contentProviderId': 1, 'contentProviderName': 'Intuit FDS', 'providerId': None}, {'contentProviderId': 2, 'contentProviderName': 'Intuit Mint Bills', 'providerId': '55'}]}, 'model': 'REGULAR_BILL', 'dueDate': '2018-03-20', 'aggregationDueAmount': 0.0, 'dueAmount': 0.0, 'currency': 'USD', 'pastDueAmount': 0.0, 'statementAmount': 0.0, 'statementDate': '2018-02-28', 'billStatus': 'FULL', 'pastDueStatus': 'FALSE', 'isPayable': False, 'isHistory': False, 'institutionType': 'LINKED', 'institutionId': '8897093450193760206', 'billDetailsList': [{'billDetailsType': 'UTILITY', 'name': 'Payments', 'value': 228.7}], 'isVisible': True, 'allowedPaymentMethodOptions': ['ACH', 'DEBIT_CARD', 'CREDIT_CARD'], 'id': '7326903310100285047', 'metaData': {'createdDate': '2017-01-13T20:44:20Z', 'link': [{'href': '/v2/payer/institutions/8897093450193760206', 'rel': 'institution'}]}}, {'contentAccountRef': {'contentProviderId': 2, 'contentProviderName': 'Intuit Mint Bills', 'contentLoginRef': '3034292192760981176', 'contentAccountRef': '75391669130', 'connectedAccounts': [{'contentProviderId': 1, 'contentProviderName': 'Intuit FDS', 'contentLoginRef': '7172b333-b65e-3872-b6f0-011be43e151a', 'contentAccountRef': 'urn:account:fdp::accountid:e594a788-a653-3e10-befe-706f2bcff320'}]}, 'providerRef': {'providerId': '75f8d952-be81-4e22-951e-41b291d211f2', 'providerName': 'American Express Corporate or Personal Credit Card (US)', 'contentProviderRef': [{'contentProviderId': 1, 'contentProviderName': 'Intuit FDS', 'providerId': '1002'}, {'contentProviderId': 2, 'contentProviderName': 'Intuit Mint Bills', 'providerId': '64'}]}, 'model': 'CREDIT_CARD', 'dueDate': '2021-07-21', 'aggregationDueAmount': 1591.23, 'dueAmount': 1591.23, 'currency': 'USD', 'statementAmount': 1591.23, 'statementDate': '2021-06-25', 'billStatus': 'NOT_PAID', 'pastDueStatus': 'FALSE', 'isPayable': False, 'lastDigits': '1003', 'minimumPaymentDue': 40.0, 'aggregationMinimumPaymentDue': 40.0, 'isHistory': False, 'institutionType': 'LINKED', 'institutionId': '3034292192760981176', 'lastPaymentDate': '2021-07-10', 'billDetailsList': [{'billDetailsType': 'CREDIT_CARD', 'name': 'Bonvoy Amex Card', 'number': '1003', 'availableBalanceAmount': 2305.6, 'availableBalanceType': 'DEBIT', 'creditLimit': 
10000.0, 'status': 'OTHER'}], 'isVisible': True, 'allowedPaymentMethodOptions': ['ACH', 'DEBIT_CARD'], 'id': '4576616495530057800', 'metaData': {'createdDate': '2019-08-03T14:23:49Z', 'link': [{'href': '/v2/payer/institutions/3034292192760981176', 'rel': 
'institution'}]}}, {'contentAccountRef': {'contentProviderId': 2, 'contentProviderName': 'Intuit Mint Bills', 'contentLoginRef': 'dce6c3b2-aced-3977-9454-bb5f58eac7bb', 'contentAccountRef': '39c2342139fcc5e34c526b87022f7773', 'connectedAccounts': [{'contentProviderId': 2, 'contentProviderName': 'Intuit Mint Bills', 'contentLoginRef': 'dce6c3b2-aced-3977-9454-bb5f58eac7bb', 'contentAccountRef': '39c2342139fcc5e34c526b87022f7773'}]}, 'providerRef': {'providerId': 'f141569e-9e1b-4ad5-a8c7-8cb3d545703d', 'providerName': 'Progressive', 'contentProviderRef': [{'contentProviderId': 1, 'contentProviderName': 'Intuit FDS', 'providerId': None}, {'contentProviderId': 2, 'contentProviderName': 'Intuit Mint Bills', 'providerId': '155'}]}, 'model': 'INSURANCE', 'dueDate': '2020-02-03', 'aggregationDueAmount': 0.0, 'dueAmount': 0.0, 'currency': 'USD', 'pastDueAmount': 0.0, 'billStatus': 'FULL', 'pastDueStatus': 'FALSE', 'isPayable': False, 'lastDigits': '6768', 'autoPaid': True, 'accountNumber': '915556768', 'paymentResetDate': '2020-01-29T05:09:48.844+00:00', 'isHistory': False, 'institutionType': 'LINKED', 'institutionId': '1489353012757094401', 'isVisible': True, 'allowedPaymentMethodOptions': ['ACH', 'DEBIT_CARD', 'CREDIT_CARD'], 'id': '2930261357954217868', 'metaData': {'createdDate': '2018-02-12T00:44:11Z', 'link': [{'href': '/v2/payer/institutions/1489353012757094401', 'rel': 'institution'}]}}, {'contentAccountRef': {'contentProviderId': 2, 'contentProviderName': 'Intuit Mint Bills', 'contentLoginRef': '7253342564148930261', 'contentAccountRef': '75370821235', 'connectedAccounts': [{'contentProviderId': 1, 'contentProviderName': 'Intuit FDS', 'contentLoginRef': '362a737e-5413-3bfb-9d20-0e164ca32442', 'contentAccountRef': 'urn:account:fdp::accountid:46d3fd5b-c157-37fe-997c-f700505f6122'}]}, 'providerRef': {'providerId': '5a386b71-bdf5-4c41-a1a5-3ad5d411cc8d', 'providerName': 'Honda Financial Services', 'contentProviderRef': [{'contentProviderId': 1, 'contentProviderName': 'Intuit FDS', 'providerId': '3755'}, {'contentProviderId': 2, 'contentProviderName': 'Intuit Mint Bills', 'providerId': '162'}]}, 'model': 'LOAN', 'dueDate': '2021-08-06', 'aggregationDueAmount': 554.16, 'dueAmount': 554.16, 'currency': 'USD', 'billStatus': 'NOT_PAID', 'pastDueStatus': 'FALSE', 'isPayable': False, 'lastDigits': '0727', 'autoPaid': True, 'accountNumber': '9719854306462430727', 'isHistory': False, 'institutionType': 'LINKED', 'institutionId': '7253342564148930261', 'lastPaymentDate': '2021-07-01', 'lastPaymentAmount': 554.16, 'billDetailsList': [{'billDetailsType': 'LOAN', 'name': 'XXXXX0727', 'currentBalance': 14683.41, 'payoffAmount': 14683.41, 'loanEndDate': '2023-11-06T16:00:00.000+00:00', 'remainingPayments': 28.0, 'loanStatus': 'OTHER'}], 'isVisible': True, 'allowedPaymentMethodOptions': ['ACH', 'DEBIT_CARD'], 'id': '1890193819581491325', 'metaData': {'createdDate': '2019-08-03T14:23:49Z', 'link': [{'href': '/v2/payer/institutions/7253342564148930261', 'rel': 'institution'}]}}, {'contentAccountRef': {'contentProviderId': 2, 'contentProviderName': 'Intuit Mint Bills', 'contentLoginRef': '1497807465002559901', 'contentAccountRef': '75180217336', 'connectedAccounts': [{'contentProviderId': 1, 'contentProviderName': 'Intuit FDS', 'contentLoginRef': 'ad959858-ca3f-31a1-8b64-35475e18e88f', 'contentAccountRef': 'urn:account:fdp::accountid:ab4438b3-08c3-3a23-9c8e-4d52320ed969'}]}, 'providerRef': {'providerId': '7e3c2c96-44da-4d23-b966-2d34f65b3401', 'providerName': 'Capital One - Credit Cards', 'contentProviderRef': [{'contentProviderId': 1, 'contentProviderName': 'Intuit FDS', 'providerId': '1407'}, {'contentProviderId': 2, 'contentProviderName': 'Intuit Mint Bills', 'providerId': '2039'}]}, 'model': 'CREDIT_CARD', 'dueDate': '2020-09-14', 'aggregationDueAmount': 59.0, 'dueAmount': 
59.0, 'currency': 'USD', 'statementAmount': 59.0, 'statementDate': '2020-08-17', 'billStatus': 'NOT_PAID', 'pastDueStatus': 'FALSE', 'isPayable': False, 'lastDigits': '5671', 'isHistory': False, 'institutionType': 'LINKED', 'institutionId': '1497807465002559901', 'lastPaymentDate': '2019-09-14', 'lastPaymentAmount': 59.0, 'billDetailsList': [{'billDetailsType': 'CREDIT_CARD', 'name': 'Venture', 'number': '5671', 'availableBalanceAmount': 59.0, 'availableBalanceType': 'DEBIT', 'availableCredit': -59.0, 'status': 'OTHER'}], 'isVisible': True, 'allowedPaymentMethodOptions': ['ACH', 'DEBIT_CARD'], 'id': '4658404481527747228', 'metaData': {'createdDate': '2019-08-03T14:23:47Z', 'link': [{'href': '/v2/payer/institutions/1497807465002559901', 'rel': 'institution'}]}}, {'contentAccountRef': {'contentProviderId': 2, 'contentProviderName': 'Intuit Mint Bills', 'contentLoginRef': '5071473431000519851', 'contentAccountRef': '75398097885', 'connectedAccounts': [{'contentProviderId': 1, 'contentProviderName': 'Intuit FDS', 'contentLoginRef': '69c3abf1-bcfe-37c2-8193-1da7ccbcae07', 'contentAccountRef': 'urn:account:fdp::accountid:4e88cf07-a7e3-3e59-9c50-b87352dd789e'}]}, 'providerRef': {'providerId': '867c4a38-0d75-46ff-a4c2-f44cb63be2c9', 'providerName': 'Chase Bank', 
'contentProviderRef': [{'contentProviderId': 1, 'contentProviderName': 'Intuit FDS', 'providerId': '4'}, {'contentProviderId': 2, 'contentProviderName': 'Intuit Mint Bills', 'providerId': '2220'}]}, 'model': 'CREDIT_CARD', 'dueDate': '2021-08-02', 'aggregationDueAmount': 353.85, 'dueAmount': 353.85, 'currency': 'USD', 'statementAmount': 353.85, 'statementDate': '2021-07-05', 'billStatus': 'NOT_PAID', 'pastDueStatus': 'FALSE', 'isPayable': False, 'lastDigits': '0995', 'minimumPaymentDue': 40.0, 'aggregationMinimumPaymentDue': 40.0, 'autoPaid': True, 'isHistory': False, 'institutionType': 'LINKED', 'institutionId': '5071473431000519851', 'lastPaymentDate': '2021-07-02', 'lastPaymentAmount': 283.93, 'billDetailsList': [{'billDetailsType': 'CREDIT_CARD', 'name': 'CREDIT CARD', 'number': '0995', 'availableBalanceAmount': 353.85, 'availableBalanceType': 'DEBIT', 'creditLimit': 13000.0, 'purchasesApr': 22.99, 'status': 'OTHER'}], 'isVisible': True, 'allowedPaymentMethodOptions': ['ACH', 'DEBIT_CARD'], 'id': '6060316510470412721', 'metaData': {'createdDate': '2019-08-03T14:23:48Z', 'link': [{'href': '/v2/payer/institutions/5071473431000519851', 'rel': 'institution'}]}}]

# investments = {"6655506":{"isinactive":"false","cost":0.0,"inception":"Jan 3, 2016","description":"Wealthfront - My Personal Investment Account","isretirement":false,"isbrokerage":false,"sorted":{"performance-asc":[],"performance-dsc":[],"value":[],"movement":[]},"dormant":false,"changeValue":0.0,"id":"6655506","investmentType":"Investment, Taxable","holdings":{},"value":0.0},"cost":55410.25000000001,"inception":"Aug 22, 2013","5721028":{"isinactive":"false","cost":0.0,"inception":"Aug 22, 2013","description":"T. Rowe  Price Retirement Plan Services - CAMERON INTERNATIONAL CORP RETIREMENT SAVINGS PLAN","isretirement":true,"isbrokerage":false,"sorted":{"performance-asc":[],"performance-dsc":[],"value":[],"movement":[]},"dormant":true,"changeValue":0.0,"id":"5721028","investmentType":"401(k)","holdings":{},"value":0.0},"6825857":{"isinactive":"false","cost":29561.52,"inception":"Jan 3, 2016","description":"Charles Schwab - Roth Contributory IRA","isretirement":true,"isbrokerage":false,"sorted":{"performance-asc":["18594792","18594798","18594795","18594797","18601923","18574960"],"performance-dsc":["18574960","18601923","18594797","18594795","18594798","18594792"],"value":["18601923","18594797","18594798","18574960","18594792","18594795"],"movement":["18574960","18601923","18594797","18594795","18594798","18594792"]},"dormant":true,"changePercent":0.6333172313196346,"changeValue":18721.820000000003,"id":"6825857","investmentType":"401(k)","holdings":{"18594792":{"symbol":"EMB","cost":224.89999999999986,"inception":"Oct 13, 2020","description":"ISHARES JPMORGAN USD MTSBOND ETF","type":15,"transactions":[{"date":"May 07","shares":null,"amount":"$0.70","category":"Dividend"},{"date":"Jun 07","shares":null,"amount":"$0.72","category":"Dividend"},{"date":"Jul 08","shares":null,"amount":"$0.71","category":"Dividend"}],"shares":2.0,"price":111.82,"changePercent":-0.005602489995553037,"changeValue":-1.2599999999998772,"id":"18594792","value":223.64,"account":"6825857"},"18594795":{"symbol":"VWO","cost":45.12000000000012,"inception":"Oct 13, 2020","description":"VANGUARD FTSE EMERGING MARK ETF IV","type":15,"transactions":[{"date":"Jun 24","shares":null,"amount":"$0.28","category":"Dividend"},{"date":"Oct 20","shares":26.0,"amount":"$1,175.17","category":"None"},{"date":"Oct 20","shares":74.0,"amount":"$3,344.73","category":"None"}],"shares":1.0,"price":53.04,"changePercent":0.17553191489361392,"changeValue":7.919999999999881,"id":"18594795","value":53.04,"account":"6825857"},"18601923":{"symbol":"TSLA","cost":19004.7,"inception":"Oct 18, 2020","description":"TESLA INC","type":15,"transactions":[{"date":"Oct 15","shares":10.0,"amount":"$4,509.30","category":"None"},{"date":"Oct 30","shares":22.0,"amount":"$8,668.00","category":"None"},{"date":"Nov 02","shares":1.0,"amount":"$396.00","category":"None"}],"shares":47.0,"price":668.54,"changePercent":0.6533478560566597,"changeValue":12416.68,"id":"18601923","value":31421.38,"account":"6825857"},"18574960":{"shares":0.0,"symbol":"","cost":0.0,"price":null,"inception":"Oct 4, 2020","description":"Cash and Money Market","changeValue":3979.23,"id":"18574960","type":15,"transactions":[],"value":3979.23,"account":"6825857"},"18594798":{"symbol":"VIG","cost":4021.8,"inception":"Oct 13, 2020","description":"VANGUARD DIVIDEND APPRECIATIO ETF IV","type":15,"transactions":[{"date":"Jun 24","shares":null,"amount":"$20.25","category":"Dividend"},{"date":"Dec 24","shares":null,"amount":"$19.93","category":"Dividend"},{"date":"Mar 25","shares":null,"amount":"$15.39","category":"Dividend"}],"shares":30.0,"price":157.41,"changePercent":0.17417574220498283,"changeValue":700.5,"id":"18594798","value":4722.3,"account":"6825857"},"18594797":{"symbol":"VTI","cost":6265.0,"inception":"Oct 13, 2020","description":"VANGUARD TOTAL STOCK MARKET E ETF IV","type":15,"transactions":[{"date":"Jun 29","shares":null,"amount":"$23.64","category":"Dividend"},{"date":"Dec 30","shares":null,"amount":"$27.36","category":"Dividend"},{"date":"Mar 29","shares":null,"amount":"$300.00","category":"None"}],"shares":35.0,"price":225.25,"changePercent":0.25837988826815644,"changeValue":1618.75,"id":"18594797","value":7883.75,"account":"6825857"}},"value":48283.340000000004},"6661189":{"isinactive":"false","cost":1962.05,"inception":"Apr 4, 2019","description":"Fidelity - INDIVIDUAL","isretirement":false,"isbrokerage":false,"sorted":{"performance-asc":["17508846"],"performance-dsc":["17508846"],"value":["17508846"],"movement":["17508846"]},"dormant":false,"changePercent":0.0,"changeValue":0.0,"id":"6661189","investmentType":"Investment, Taxable","holdings":{"17508846":{"symbol":"SPAXX","cost":1962.05,"inception":"Apr 4, 2019","description":"FIDELITY GOVERNMENT MONEY MARKET","type":15,"transactions":[],"shares":1962.05,"price":1.0,"changePercent":0.0,"changeValue":0.0,"id":"17508846","value":1962.05,"account":"6661189"}},"value":1962.05},"sorted":{"performance-asc":["6825857-18594792","6216793-15771339","6216793-15771335","6661189-17508846","6216793-15771341","6825857-18594798","6825857-18594795","6825857-18594797","6834413-186417218883906813,"changeValue":13683.54,"id":"18641721","value":35432.62,"account":"6834413"},"18613735":{"shares":0.0,"symbol":"","cost":0.0,"price":null,"inception":"Oct 24, 2020","description":"Cash and Money Market","changeValue":4.76,"id":"18613735","type":15,"transactions":[],"value":4.76,"account":"6834413"}},"value":35437.380000000005},"segment":"multiple","showPopup":null,"changePercent":1.117761425007106,"changeValue":61935.439999999995,"value":117345.69,"6216793":{"isinactive":"false","cost":0.0,"inception":"Jan 3, 2016","description":"Wealthfront - My Roth IRA","isretirement":false,"isbrokerage":false,"sorted":{"performance-asc":["15771339","15771335","15771341"],"performance-dsc":["15771335","15771341","15771339"],"value":["15771341","15771335","15771339"],"movement":["15771339","15771335","15771341"]},"dormant":false,"changeValue":0.0,"id":"6216793","investmentType":"Investment, Taxable","holdings":{"15771339":{"symbol":"","cost":-7.77,"inception":"Feb 3, 2018","description":"Cash","type":15,"transactions":[],"shares":0.0,"price":null,"changePercent":-0.0,"changeValue":0.0,"id":"15771339","value":-7.77,"account":"6216793"},"15771335":{"symbol":"","cost":2.65,"inception":"Feb 3, 2018","description":"Corporate Bonds","type":15,"transactions":[],"shares":0.0,"price":null,"changePercent":0.0,"changeValue":0.0,"id":"15771335","value":2.65,"account":"6216793"},"15771341":{"symbol":"","cost":5.12,"inception":"Feb 3, 2018","description":"Emerging Market Bonds","type":15,"transactions":[],"shares":0.0,"price":null,"changePercent":0.0,"changeValue":0.0,"id":"15771341","value":5.12,"account":"6216793"}},"value":0.0},"lastDate":"Wed Jul 14 close"}



In [2]:
r_investments = """{"6655506":{"isinactive":"false","cost":0.0,"inception":"Jan 3, 2016","description":"Wealthfront - My Personal Investment Account","isretirement":false,"isbrokerage":false,"sorted":{"performance-asc":[],"performance-dsc":[],"value":[],"movement":[]},"dormant":false,"changeValue":0.0,"id":"6655506","investmentType":"Investment, Taxable","holdings":{},"value":0.0},"cost":55410.25000000001,"inception":"Aug 22, 2013","5721028":{"isinactive":"false","cost":0.0,"inception":"Aug 22, 2013","description":"T. Rowe Price Retirement Plan Services - CAMERON INTERNATIONAL CORP RETIREMENT SAVINGS PLAN","isretirement":true,"isbrokerage":false,"sorted":{"performance-asc":[],"performance-dsc":[],"value":[],"movement":[]},"dormant":true,"changeValue":0.0,"id":"5721028","investmentType":"401(k)","holdings":{},"value":0.0},"6825857":{"isinactive":"false","cost":29561.520000000004,"inception":"Jan 3, 2016","description":"Charles Schwab - Roth Contributory IRA","isretirement":true,"isbrokerage":false,"sorted":{"performance-asc":["18594792","18594798","18594795","18594797","18601923","18574960"],"performance-dsc":["18574960","18601923","18594797","18594795","18594798","18594792"],"value":["18601923","18594797","18594798","18574960","18594792","18594795"],"movement":["18574960","18601923","18594797","18594795","18594798","18594792"]},"dormant":true,"changePercent":0.6333172313196344,"changeValue":18721.82,"id":"6825857","investmentType":"401(k)","holdings":{"18594792":{"symbol":"EMB","cost":224.89999999999986,"inception":"Oct 13, 2020","description":"ISHARES JPMORGAN USD MTSBOND ETF","type":15,"transactions":[{"date":"May 07","shares":null,"amount":"$0.70","category":"Dividend"},{"date":"Jun 07","shares":null,"amount":"$0.72","category":"Dividend"},{"date":"Jul 08","shares":null,"amount":"$0.71","category":"Dividend"}],"shares":2.0,"price":111.82,"changePercent":-0.005602489995553037,"changeValue":-1.2599999999998772,"id":"18594792","value":223.64,"account":"6825857"},"18594795":{"symbol":"VWO","cost":45.12000000000012,"inception":"Oct 13, 2020","description":"VANGUARD FTSE EMERGING MARK ETF IV","type":15,"transactions":[{"date":"Jun 24","shares":null,"amount":"$0.28","category":"Dividend"},{"date":"Oct 20","shares":26.0,"amount":"$1,175.17","category":"None"},{"date":"Oct 20","shares":74.0,"amount":"$3,344.73","category":"None"}],"shares":1.0,"price":53.04,"changePercent":0.17553191489361392,"changeValue":7.919999999999881,"id":"18594795","value":53.04,"account":"6825857"},"18601923":{"symbol":"TSLA","cost":19004.7,"inception":"Oct 18, 2020","description":"TESLA INC","type":15,"transactions":[{"date":"Oct 15","shares":10.0,"amount":"$4,509.30","category":"None"},{"date":"Oct 30","shares":22.0,"amount":"$8,668.00","category":"None"},{"date":"Nov 02","shares":1.0,"amount":"$396.00","category":"None"}],"shares":47.0,"price":668.54,"changePercent":0.6533478560566597,"changeValue":12416.68,"id":"18601923","value":31421.38,"account":"6825857"},"18594798":{"symbol":"VIG","cost":4021.8,"inception":"Oct 13, 2020","description":"VANGUARD DIVIDEND APPRECIATIO ETF IV","type":15,"transactions":[{"date":"Jun 24","shares":null,"amount":"$20.25","category":"Dividend"},{"date":"Dec 24","shares":null,"amount":"$19.93","category":"Dividend"},{"date":"Mar 25","shares":null,"amount":"$15.39","category":"Dividend"}],"shares":30.0,"price":157.41,"changePercent":0.17417574220498283,"changeValue":700.5,"id":"18594798","value":4722.3,"account":"6825857"},"18574960":{"shares":0.0,"symbol":"","cost":0.0,"price":null,"inception":"Oct 4, 2020","description":"Cash and Money Market","changeValue":3979.23,"id":"18574960","type":15,"transactions":[],"value":3979.23,"account":"6825857"},"18594797":{"symbol":"VTI","cost":6265.0,"inception":"Oct 13, 2020","description":"VANGUARD TOTAL STOCK MARKET E ETF IV","type":15,"transactions":[{"date":"Jun 29","shares":null,"amount":"$23.64","category":"Dividend"},{"date":"Dec 30","shares":null,"amount":"$27.36","category":"Dividend"},{"date":"Mar 29","shares":null,"amount":"$300.00","category":"None"}],"shares":35.0,"price":225.25,"changePercent":0.25837988826815644,"changeValue":1618.75,"id":"18594797","value":7883.75,"account":"6825857"}},"value":48283.340000000004},"6661189":{"isinactive":"false","cost":1962.05,"inception":"Apr 4, 2019","description":"Fidelity - INDIVIDUAL","isretirement":false,"isbrokerage":false,"sorted":{"performance-asc":["17508846"],"performance-dsc":["17508846"],"value":["17508846"],"movement":["17508846"]},"dormant":false,"changePercent":0.0,"changeValue":0.0,"id":"6661189","investmentType":"Investment, Taxable","holdings":{"17508846":{"symbol":"SPAXX","cost":1962.05,"inception":"Apr 4, 2019","description":"FIDELITY GOVERNMENT MONEY MARKET","type":15,"transactions":[],"shares":1962.05,"price":1.0,"changePercent":0.0,"changeValue":0.0,"id":"17508846","value":1962.05,"account":"6661189"}},"value":1962.05},"sorted":{"performance-asc":["6825857-18594792","6216793-15771339","6216793-15771335","6661189-17508846","6216793-15771341","6825857-18594798","6825857-18594795","6825857-18594797","6834413-18641721","6825857-18601923","6444597-14933658","6444597-14933659","6825857-18574960","6834413-18613735"],"performance-dsc":["6444597-14933659","6825857-18574960","6834413-18613735","6444597-14933658","6825857-18601923","6834413-18641721","6825857-18594797","6825857-18594795","6825857-18594798","6216793-15771335","6661189-17508846","6216793-15771341","6216793-15771339","6825857-18594792"],"value":["6834413-18641721","6825857-18601923","6444597-14933658","6825857-18594797","6825857-18594798","6825857-18574960","6444597-14933659","6661189-17508846","6825857-18594792","6825857-18594795","6216793-15771341","6834413-18613735","6216793-15771335","6216793-15771339"],"movement":["6444597-14933659","6825857-18574960","6834413-18613735","6444597-14933658","6825857-18601923","6834413-18641721","6825857-18594797","6825857-18594795","6825857-18594798","6825857-18594792","6216793-15771339","6216793-15771335","6661189-17508846","6216793-15771341"]},"6444597":{"isinactive":"false","cost":2137.6,"inception":"Jan 3, 2016","description":"Charles Schwab - Individual                       ","isretirement":false,"isbrokerage":false,"sorted":{"performance-asc":["14933658","14933659"],"performance-dsc":["14933659","14933658"],"value":["14933658","14933659"],"movement":["14933659","14933658"]},"dormant":false,"changePercent":13.81236901197605,"changeValue":29525.320000000003,"id":"6444597","investmentType":"Investment, Taxable","holdings":{"14933658":{"symbol":"TSLA","cost":2137.6,"inception":"Jun 29, 2017","description":"TESLA INC","type":15,"transactions":[{"date":"Sep 23","shares":20.0,"amount":"$8,000.00","category":"None"},{"date":"Nov 10","shares":3.0,"amount":"$1,245.00","category":"None"}],"shares":43.0,"price":668.54,"changePercent":12.4483626497006,"changeValue":26609.620000000003,"id":"14933658","value":28747.22,"account":"6444597"},"14933659":{"shares":0.0,"symbol":"","cost":0.0,"price":null,"inception":"Jun 29, 2017","description":"Cash and Money Market","changeValue":2915.7,"id":"14933659","type":15,"transactions":[],"value":2915.7,"account":"6444597"}},"value":31662.920000000002},"6474862":{"isinactive":"false","dormant":false,"inception":"Sep 27, 2017","description":"Acorns - Acorns","id":"6474862","investmentType":"Investment, Taxable","isretirement":false,"isbrokerage":false},"6834413":{"isinactive":"false","cost":21749.08,"inception":"Jan 3, 2016","description":"Charles Schwab - HSA Brokerage","isretirement":true,"isbrokerage":false,"sorted":{"performance-asc":["18641721","18613735"],"performance-dsc":["18613735","18641721"],"value":["18641721","18613735"],"movement":["18613735","18641721"]},"dormant":true,"changePercent":0.6293737482229134,"changeValue":13688.300000000003,"id":"6834413","investmentType":"401(k)","holdings":{"18641721":{"symbol":"TSLA","cost":21749.08,"inception":"Nov 10, 2020","description":"TESLA INC","type":15,"transactions":[{"date":"Nov 10","shares":53.0,"amount":"$21,836.00","category":"None"}],"shares":53.0,"price":668.54,"changePercent":0.6291548883906813,"changeValue":13683.54,"id":"18641721","value":35432.62,"account":"6834413"},"18613735":{"shares":0.0,"symbol":"","cost":0.0,"price":null,"inception":"Oct 24, 2020","description":"Cash and Money Market","changeValue":4.76,"id":"18613735","type":15,"transactions":[],"value":4.76,"account":"6834413"}},"value":35437.380000000005},"segment":"multiple","showPopup":null,"changePercent":1.1177614250071062,"changeValue":61935.44000000001,"value":117345.69000000002,"6216793":{"isinactive":"false","cost":0.0,"inception":"Jan 3, 2016","description":"Wealthfront - My Roth IRA","isretirement":false,"isbrokerage":false,"sorted":{"performance-asc":["15771339","15771335","15771341"],"performance-dsc":["15771335","15771341","15771339"],"value":["15771341","15771335","15771339"],"movement":["15771339","15771335","15771341"]},"dormant":false,"changeValue":4.440892098500626E-16,"id":"6216793","investmentType":"Investment, Taxable","holdings":{"15771339":{"symbol":"","cost":-7.77,"inception":"Feb 3, 2018","description":"Cash","type":15,"transactions":[],"shares":0.0,"price":null,"changePercent":-0.0,"changeValue":0.0,"id":"15771339","value":-7.77,"account":"6216793"},"15771335":{"symbol":"","cost":2.65,"inception":"Feb 3, 2018","description":"Corporate Bonds","type":15,"transactions":[],"shares":0.0,"price":null,"changePercent":0.0,"changeValue":0.0,"id":"15771335","value":2.65,"account":"6216793"},"15771341":{"symbol":"","cost":5.12,"inception":"Feb 3, 2018","description":"Emerging Market Bonds","type":15,"transactions":[],"shares":0.0,"price":null,"changePercent":0.0,"changeValue":0.0,"id":"15771341","value":5.12,"account":"6216793"}},"value":4.440892098500626E-16},"lastDate":"Wed Jul 14 close"}"""

In [58]:
info = json.loads(r_investments)
pp.pprint(info)

{'5721028': {'changeValue': 0.0,
             'cost': 0.0,
             'description': 'T. Rowe Price Retirement Plan Services - CAMERON '
                            'INTERNATIONAL CORP RETIREMENT SAVINGS PLAN',
             'dormant': True,
             'holdings': {},
             'id': '5721028',
             'inception': 'Aug 22, 2013',
             'investmentType': '401(k)',
             'isbrokerage': False,
             'isinactive': 'false',
             'isretirement': True,
             'sorted': {'movement': [],
                        'performance-asc': [],
                        'performance-dsc': [],
                        'value': []},
             'value': 0.0},
 '6216793': {'changeValue': 4.440892098500626e-16,
             'cost': 0.0,
             'description': 'Wealthfront - My Roth IRA',
             'dormant': False,
             'holdings': {'15771335': {'account': '6216793',
                                       'changePercent': 0.0,
                      

In [9]:
print(info.keys())

dict_keys(['6655506', 'cost', 'inception', '5721028', '6825857', '6661189', 'sorted', '6444597', '6474862', '6834413', 'segment', 'showPopup', 'changePercent', 'changeValue', 'value', '6216793', 'lastDate'])


In [19]:
info['6655506']

{'isinactive': 'false',
 'cost': 0.0,
 'inception': 'Jan 3, 2016',
 'description': 'Wealthfront - My Personal Investment Account',
 'isretirement': False,
 'isbrokerage': False,
 'sorted': {'performance-asc': [],
  'performance-dsc': [],
  'value': [],
  'movement': []},
 'dormant': False,
 'changeValue': 0.0,
 'id': '6655506',
 'investmentType': 'Investment, Taxable',
 'holdings': {},
 'value': 0.0}

In [91]:
def get_investmentAccount_Nums(info_dict):
    account_numbers = set()

    for key in info_dict.keys():
        if key.isnumeric():
            account_numbers.add(key)
    
    print(account_numbers)
    
    return account_numbers


In [92]:

account_nums =  get_investmentAccount_Nums(info)

account_details = {key:value for key, value in info.items() if key in account_nums}

pp.pprint(account_details)

{'6661189', '6825857', '6834413', '6655506', '6474862', '5721028', '6216793', '6444597'}
{'5721028': {'changeValue': 0.0,
             'cost': 0.0,
             'description': 'T. Rowe Price Retirement Plan Services - CAMERON '
                            'INTERNATIONAL CORP RETIREMENT SAVINGS PLAN',
             'dormant': True,
             'holdings': {},
             'id': '5721028',
             'inception': 'Aug 22, 2013',
             'investmentType': '401(k)',
             'isbrokerage': False,
             'isinactive': 'false',
             'isretirement': True,
             'sorted': {'movement': [],
                        'performance-asc': [],
                        'performance-dsc': [],
                        'value': []},
             'value': 0.0},
 '6216793': {'changeValue': 4.440892098500626e-16,
             'cost': 0.0,
             'description': 'Wealthfront - My Roth IRA',
             'dormant': False,
             'holdings': {'15771335': {'account': '6216

In [118]:
account_details['6444597']['holdings']['14933658']

{'symbol': 'TSLA',
 'cost': 2137.6,
 'inception': 'Jun 29, 2017',
 'description': 'TESLA INC',
 'type': 15,
 'transactions': [{'date': 'Sep 23',
   'shares': 20.0,
   'amount': '$8,000.00',
   'category': 'None'},
  {'date': 'Nov 10',
   'shares': 3.0,
   'amount': '$1,245.00',
   'category': 'None'}],
 'shares': 43.0,
 'price': 668.54,
 'changePercent': 12.4483626497006,
 'changeValue': 26609.620000000003,
 'id': '14933658',
 'value': 28747.22,
 'account': '6444597'}

In [124]:
account_details['6444597']

{'isinactive': 'false',
 'cost': 2137.6,
 'inception': 'Jan 3, 2016',
 'description': 'Charles Schwab - Individual                       ',
 'isretirement': False,
 'isbrokerage': False,
 'sorted': {'performance-asc': ['14933658', '14933659'],
  'performance-dsc': ['14933659', '14933658'],
  'value': ['14933658', '14933659'],
  'movement': ['14933659', '14933658']},
 'dormant': False,
 'changePercent': 13.81236901197605,
 'changeValue': 29525.320000000003,
 'id': '6444597',
 'investmentType': 'Investment, Taxable',
 'holdings': {'14933658': {'symbol': 'TSLA',
   'cost': 2137.6,
   'inception': 'Jun 29, 2017',
   'description': 'TESLA INC',
   'type': 15,
   'transactions': [{'date': 'Sep 23',
     'shares': 20.0,
     'amount': '$8,000.00',
     'category': 'None'},
    {'date': 'Nov 10',
     'shares': 3.0,
     'amount': '$1,245.00',
     'category': 'None'}],
   'shares': 43.0,
   'price': 668.54,
   'changePercent': 12.4483626497006,
   'changeValue': 26609.620000000003,
   'id': '

In [125]:
new_df = pd.DataFrame(account_details['6444597'])

In [109]:
# {'6661189', '6825857', '6834413', '6655506', '6474862', '5721028', '6216793', '6444597'}
df_test = pd.DataFrame()

# df_test


df_1 = pd.DataFrame(account_details['6444597'])

print(df_1.shape)
df_test = df_1

df_2 = pd.DataFrame(account_details['5721028'])
df_test.append(df_2)

df_3 = pd.DataFrame(account_details['6661189'])
df_test.append(df_3)

df_4 = pd.DataFrame(account_details['6825857'])
df_test.append(df_4)

df_5 = pd.DataFrame(account_details['6834413'])
df_test.append(df_5)

df_2 = pd.DataFrame(account_details['5721028'])
df_test.append(df_1)

df_2 = pd.DataFrame(account_details['5721028'])
df_test.append(df_1)

# df_2 = pd.DataFrame(account_details['5721028'])

print(df_2.shape)
# df_test.append(df_2)
df_test.shape
# print(df_test.head())

(6, 14)
(4, 13)


(6, 14)

In [126]:
acc_572 = nested_to_record(account_details['6444597'],sep='_')
acc_572


{'isinactive': 'false',
 'cost': 2137.6,
 'inception': 'Jan 3, 2016',
 'description': 'Charles Schwab - Individual                       ',
 'isretirement': False,
 'isbrokerage': False,
 'dormant': False,
 'changePercent': 13.81236901197605,
 'changeValue': 29525.320000000003,
 'id': '6444597',
 'investmentType': 'Investment, Taxable',
 'value': 31662.920000000002,
 'sorted_performance-asc': ['14933658', '14933659'],
 'sorted_performance-dsc': ['14933659', '14933658'],
 'sorted_value': ['14933658', '14933659'],
 'sorted_movement': ['14933659', '14933658'],
 'holdings_14933658_symbol': 'TSLA',
 'holdings_14933658_cost': 2137.6,
 'holdings_14933658_inception': 'Jun 29, 2017',
 'holdings_14933658_description': 'TESLA INC',
 'holdings_14933658_type': 15,
 'holdings_14933658_transactions': [{'date': 'Sep 23',
   'shares': 20.0,
   'amount': '$8,000.00',
   'category': 'None'},
  {'date': 'Nov 10',
   'shares': 3.0,
   'amount': '$1,245.00',
   'category': 'None'}],
 'holdings_14933658_shar

In [76]:
new_df = pd.DataFrame()

for key, value in account_details.items():
    sample_df = pd.DataFrame(value)
    print(len(value))
    
    print('\n\n\n')
    new_df.append(sample_df)

new_df

13




13




14




14




14






ValueError: If using all scalar values, you must pass an index

In [56]:
info_flat = nested_to_record(account_details, sep='_')

In [57]:
pp.pprint(info_flat)

{0: {'5721028_changeValue': 0.0,
     '5721028_cost': 0.0,
     '5721028_description': 'T. Rowe Price Retirement Plan Services - CAMERON '
                            'INTERNATIONAL CORP RETIREMENT SAVINGS PLAN',
     '5721028_dormant': True,
     '5721028_id': '5721028',
     '5721028_inception': 'Aug 22, 2013',
     '5721028_investmentType': '401(k)',
     '5721028_isbrokerage': False,
     '5721028_isinactive': 'false',
     '5721028_isretirement': True,
     '5721028_sorted_movement': [],
     '5721028_sorted_performance-asc': [],
     '5721028_sorted_performance-dsc': [],
     '5721028_sorted_value': [],
     '5721028_value': 0.0,
     '6216793_changeValue': 4.440892098500626e-16,
     '6216793_cost': 0.0,
     '6216793_description': 'Wealthfront - My Roth IRA',
     '6216793_dormant': False,
     '6216793_holdings_15771335_account': '6216793',
     '6216793_holdings_15771335_changePercent': 0.0,
     '6216793_holdings_15771335_changeValue': 0.0,
     '6216793_holdings_15771335_co

 '6834413_holdings_18613735_account': '6834413',
 '6834413_holdings_18613735_changeValue': 4.76,
 '6834413_holdings_18613735_cost': 0.0,
 '6834413_holdings_18613735_description': 'Cash and Money Market',
 '6834413_holdings_18613735_id': '18613735',
 '6834413_holdings_18613735_inception': 'Oct 24, 2020',
 '6834413_holdings_18613735_price': None,
 '6834413_holdings_18613735_shares': 0.0,
 '6834413_holdings_18613735_symbol': '',
 '6834413_holdings_18613735_transactions': [],
 '6834413_holdings_18613735_type': 15,
 '6834413_holdings_18613735_value': 4.76,
 '6834413_holdings_18641721_account': '6834413',
 '6834413_holdings_18641721_changePercent': 0.6291548883906813,
 '6834413_holdings_18641721_changeValue': 13683.54,
 '6834413_holdings_18641721_cost': 21749.08,
 '6834413_holdings_18641721_description': 'TESLA INC',
 '6834413_holdings_18641721_id': '18641721',
 '6834413_holdings_18641721_inception': 'Nov 10, 2020',
 '6834413_holdings_18641721_price': 668.54,
 '6834413_holdings_18641721_shar

In [27]:
pp.pprint(info['6444597'])

{'changePercent': 13.81236901197605,
 'changeValue': 29525.320000000003,
 'cost': 2137.6,
 'description': 'Charles Schwab - Individual                       ',
 'dormant': False,
 'holdings': {'14933658': {'account': '6444597',
                           'changePercent': 12.4483626497006,
                           'changeValue': 26609.620000000003,
                           'cost': 2137.6,
                           'description': 'TESLA INC',
                           'id': '14933658',
                           'inception': 'Jun 29, 2017',
                           'price': 668.54,
                           'shares': 43.0,
                           'symbol': 'TSLA',
                           'transactions': [{'amount': '$8,000.00',
                                             'category': 'None',
                                             'date': 'Sep 23',
                                             'shares': 20.0},
                                            {'amount': '$1,

In [127]:
info = json.loads(r_investments)



investments_df = pd.DataFrame(info)

investments_df

,6655506,cost,inception,5721028,6825857,6661189,sorted,6444597,6474862,6834413,segment,showPopup,changePercent,changeValue,value,6216793,lastDate
isinactive,false,55410.25,"Aug 22, 2013",false,false,false,NaN,false,false,false,multiple,None,1.117761,61935.44,117345.69,false,Wed Jul 14 close
cost,0,55410.25,"Aug 22, 2013",0,29561.5,1962.05,NaN,2137.6,NaN,21749.1,multiple,None,1.117761,61935.44,117345.69,0,Wed Jul 14 close
inception,"Jan 3, 2016",55410.25,"Aug 22, 2013","Aug 22, 2013","Jan 3, 2016","Apr 4, 2019",NaN,"Jan 3, 2016","Sep 27, 2017","Jan 3, 2016",multiple,None,1.117761,61935.44,117345.69,"Jan 3, 2016",Wed Jul 14 close
description,Wealthfront - My Personal Investment Account,55410.25,"Aug 22, 2013",T. Rowe Price Retirement Plan Services - CAMER...,Charles Schwab - Roth Contributory IRA,Fidelity - INDIVIDUAL,NaN,Charles Schwab - Individual ...,Acorns - Acorns,Charles Schwab - HSA Brokerage,multiple,None,1.117761,61935.44,117345.69,Wealthfront - My Roth IRA,Wed Jul 14 close
isretirement,False,55410.25,"Aug 22, 2013",True,True,False,NaN,False,False,True,multiple,None,1.117761,61935.44,117345.69,False,Wed Jul 14 close
isbrokerage,False,55410.25,"Aug 22, 2013",False,False,False,NaN,False,False,False,multiple,None,1.117761,61935.44,117345.69,False,Wed Jul 14 close
sorted,"{'performance-asc': [], 'performance-dsc': [],...",55410.25,"Aug 22, 2013","{'performance-asc': [], 'performance-dsc': [],...","{'performance-asc': ['18594792', '18594798', '...","{'performance-asc': ['17508846'], 'performance...",NaN,"{'performance-asc': ['14933658', '14933659'], ...",NaN,"{'performance-asc': ['18641721', '18613735'], ...",multiple,None,1.117761,61935.44,117345.69,"{'performance-asc': ['15771339', '15771335', '...",Wed Jul 14 close
dormant,False,55410.25,"Aug 22, 2013",True,True,False,NaN,False,False,True,multiple,None,1.117761,61935.44,117345.69,False,Wed Jul 14 close
changeValue,0,55410.25,"Aug 22, 2013",0,18721.8,0,NaN,29525.3,NaN,13688.3,multiple,None,1.117761,61935.44,117345.69,4.44089e-16,Wed Jul 14 close
id,6655506,55410.25,"Aug 22, 2013",5721028,6825857,6661189,NaN,6444597,6474862,6834413,multiple,None,1.117761,61935.44,117345.69,6216793,Wed Jul 14 close


In [4]:
info


{'6655506': {'isinactive': 'false',
  'cost': 0.0,
  'inception': 'Jan 3, 2016',
  'description': 'Wealthfront - My Personal Investment Account',
  'isretirement': False,
  'isbrokerage': False,
  'sorted': {'performance-asc': [],
   'performance-dsc': [],
   'value': [],
   'movement': []},
  'dormant': False,
  'changeValue': 0.0,
  'id': '6655506',
  'investmentType': 'Investment, Taxable',
  'holdings': {},
  'value': 0.0},
 'cost': 55410.25000000001,
 'inception': 'Aug 22, 2013',
 '5721028': {'isinactive': 'false',
  'cost': 0.0,
  'inception': 'Aug 22, 2013',
  'description': 'T. Rowe Price Retirement Plan Services - CAMERON INTERNATIONAL CORP RETIREMENT SAVINGS PLAN',
  'isretirement': True,
  'isbrokerage': False,
  'sorted': {'performance-asc': [],
   'performance-dsc': [],
   'value': [],
   'movement': []},
  'dormant': True,
  'changeValue': 0.0,
  'id': '5721028',
  'investmentType': '401(k)',
  'holdings': {},
  'value': 0.0},
 '6825857': {'isinactive': 'false',
  'cost':

In [10]:
# convert to string
input = json.dumps({"6655506":{"isinactive":"false","cost":0.0,"inception":"Jan 3, 2016","description":"Wealthfront - My Personal Investment Account","isretirement":false,"isbrokerage":false,"sorted":{"performance-asc":[],"performance-dsc":[],"value":[],"movement":[]},"dormant":false,"changeValue":0.0,"id":"6655506","investmentType":"Investment, Taxable","holdings":{},"value":0.0},"cost":55410.25000000001,"inception":"Aug 22, 2013","5721028":{"isinactive":"false","cost":0.0,"inception":"Aug 22, 2013","description":"T. Rowe Price Retirement Plan Services - CAMERON INTERNATIONAL CORP RETIREMENT SAVINGS PLAN","isretirement":true,"isbrokerage":false,"sorted":{"performance-asc":[],"performance-dsc":[],"value":[],"movement":[]},"dormant":true,"changeValue":0.0,"id":"5721028","investmentType":"401(k)","holdings":{},"value":0.0},"6825857":{"isinactive":"false","cost":29561.520000000004,"inception":"Jan 3, 2016","description":"Charles Schwab - Roth Contributory IRA","isretirement":true,"isbrokerage":false,"sorted":{"performance-asc":["18594792","18594798","18594795","18594797","18601923","18574960"],"performance-dsc":["18574960","18601923","18594797","18594795","18594798","18594792"],"value":["18601923","18594797","18594798","18574960","18594792","18594795"],"movement":["18574960","18601923","18594797","18594795","18594798","18594792"]},"dormant":true,"changePercent":0.6333172313196344,"changeValue":18721.82,"id":"6825857","investmentType":"401(k)","holdings":{"18594792":{"symbol":"EMB","cost":224.89999999999986,"inception":"Oct 13, 2020","description":"ISHARES JPMORGAN USD MTSBOND ETF","type":15,"transactions":[{"date":"May 07","shares":null,"amount":"$0.70","category":"Dividend"},{"date":"Jun 07","shares":null,"amount":"$0.72","category":"Dividend"},{"date":"Jul 08","shares":null,"amount":"$0.71","category":"Dividend"}],"shares":2.0,"price":111.82,"changePercent":-0.005602489995553037,"changeValue":-1.2599999999998772,"id":"18594792","value":223.64,"account":"6825857"},"18594795":{"symbol":"VWO","cost":45.12000000000012,"inception":"Oct 13, 2020","description":"VANGUARD FTSE EMERGING MARK ETF IV","type":15,"transactions":[{"date":"Jun 24","shares":null,"amount":"$0.28","category":"Dividend"},{"date":"Oct 20","shares":26.0,"amount":"$1,175.17","category":"None"},{"date":"Oct 20","shares":74.0,"amount":"$3,344.73","category":"None"}],"shares":1.0,"price":53.04,"changePercent":0.17553191489361392,"changeValue":7.919999999999881,"id":"18594795","value":53.04,"account":"6825857"},"18601923":{"symbol":"TSLA","cost":19004.7,"inception":"Oct 18, 2020","description":"TESLA INC","type":15,"transactions":[{"date":"Oct 15","shares":10.0,"amount":"$4,509.30","category":"None"},{"date":"Oct 30","shares":22.0,"amount":"$8,668.00","category":"None"},{"date":"Nov 02","shares":1.0,"amount":"$396.00","category":"None"}],"shares":47.0,"price":668.54,"changePercent":0.6533478560566597,"changeValue":12416.68,"id":"18601923","value":31421.38,"account":"6825857"},"18594798":{"symbol":"VIG","cost":4021.8,"inception":"Oct 13, 2020","description":"VANGUARD DIVIDEND APPRECIATIO ETF IV","type":15,"transactions":[{"date":"Jun 24","shares":null,"amount":"$20.25","category":"Dividend"},{"date":"Dec 24","shares":null,"amount":"$19.93","category":"Dividend"},{"date":"Mar 25","shares":null,"amount":"$15.39","category":"Dividend"}],"shares":30.0,"price":157.41,"changePercent":0.17417574220498283,"changeValue":700.5,"id":"18594798","value":4722.3,"account":"6825857"},"18574960":{"shares":0.0,"symbol":"","cost":0.0,"price":null,"inception":"Oct 4, 2020","description":"Cash and Money Market","changeValue":3979.23,"id":"18574960","type":15,"transactions":[],"value":3979.23,"account":"6825857"},"18594797":{"symbol":"VTI","cost":6265.0,"inception":"Oct 13, 2020","description":"VANGUARD TOTAL STOCK MARKET E ETF IV","type":15,"transactions":[{"date":"Jun 29","shares":null,"amount":"$23.64","category":"Dividend"},{"date":"Dec 30","shares":null,"amount":"$27.36","category":"Dividend"},{"date":"Mar 29","shares":null,"amount":"$300.00","category":"None"}],"shares":35.0,"price":225.25,"changePercent":0.25837988826815644,"changeValue":1618.75,"id":"18594797","value":7883.75,"account":"6825857"}},"value":48283.340000000004},"6661189":{"isinactive":"false","cost":1962.05,"inception":"Apr 4, 2019","description":"Fidelity - INDIVIDUAL","isretirement":false,"isbrokerage":false,"sorted":{"performance-asc":["17508846"],"performance-dsc":["17508846"],"value":["17508846"],"movement":["17508846"]},"dormant":false,"changePercent":0.0,"changeValue":0.0,"id":"6661189","investmentType":"Investment, Taxable","holdings":{"17508846":{"symbol":"SPAXX","cost":1962.05,"inception":"Apr 4, 2019","description":"FIDELITY GOVERNMENT MONEY MARKET","type":15,"transactions":[],"shares":1962.05,"price":1.0,"changePercent":0.0,"changeValue":0.0,"id":"17508846","value":1962.05,"account":"6661189"}},"value":1962.05},"sorted":{"performance-asc":["6825857-18594792","6216793-15771339","6216793-15771335","6661189-17508846","6216793-15771341","6825857-18594798","6825857-18594795","6825857-18594797","6834413-18641721","6825857-18601923","6444597-14933658","6444597-14933659","6825857-18574960","6834413-18613735"],"performance-dsc":["6444597-14933659","6825857-18574960","6834413-18613735","6444597-14933658","6825857-18601923","6834413-18641721","6825857-18594797","6825857-18594795","6825857-18594798","6216793-15771335","6661189-17508846","6216793-15771341","6216793-15771339","6825857-18594792"],"value":["6834413-18641721","6825857-18601923","6444597-14933658","6825857-18594797","6825857-18594798","6825857-18574960","6444597-14933659","6661189-17508846","6825857-18594792","6825857-18594795","6216793-15771341","6834413-18613735","6216793-15771335","6216793-15771339"],"movement":["6444597-14933659","6825857-18574960","6834413-18613735","6444597-14933658","6825857-18601923","6834413-18641721","6825857-18594797","6825857-18594795","6825857-18594798","6825857-18594792","6216793-15771339","6216793-15771335","6661189-17508846","6216793-15771341"]},"6444597":{"isinactive":"false","cost":2137.6,"inception":"Jan 3, 2016","description":"Charles Schwab - Individual                       ","isretirement":false,"isbrokerage":false,"sorted":{"performance-asc":["14933658","14933659"],"performance-dsc":["14933659","14933658"],"value":["14933658","14933659"],"movement":["14933659","14933658"]},"dormant":false,"changePercent":13.81236901197605,"changeValue":29525.320000000003,"id":"6444597","investmentType":"Investment, Taxable","holdings":{"14933658":{"symbol":"TSLA","cost":2137.6,"inception":"Jun 29, 2017","description":"TESLA INC","type":15,"transactions":[{"date":"Sep 23","shares":20.0,"amount":"$8,000.00","category":"None"},{"date":"Nov 10","shares":3.0,"amount":"$1,245.00","category":"None"}],"shares":43.0,"price":668.54,"changePercent":12.4483626497006,"changeValue":26609.620000000003,"id":"14933658","value":28747.22,"account":"6444597"},"14933659":{"shares":0.0,"symbol":"","cost":0.0,"price":null,"inception":"Jun 29, 2017","description":"Cash and Money Market","changeValue":2915.7,"id":"14933659","type":15,"transactions":[],"value":2915.7,"account":"6444597"}},"value":31662.920000000002},"6474862":{"isinactive":"false","dormant":false,"inception":"Sep 27, 2017","description":"Acorns - Acorns","id":"6474862","investmentType":"Investment, Taxable","isretirement":false,"isbrokerage":false},"6834413":{"isinactive":"false","cost":21749.08,"inception":"Jan 3, 2016","description":"Charles Schwab - HSA Brokerage","isretirement":true,"isbrokerage":false,"sorted":{"performance-asc":["18641721","18613735"],"performance-dsc":["18613735","18641721"],"value":["18641721","18613735"],"movement":["18613735","18641721"]},"dormant":true,"changePercent":0.6293737482229134,"changeValue":13688.300000000003,"id":"6834413","investmentType":"401(k)","holdings":{"18641721":{"symbol":"TSLA","cost":21749.08,"inception":"Nov 10, 2020","description":"TESLA INC","type":15,"transactions":[{"date":"Nov 10","shares":53.0,"amount":"$21,836.00","category":"None"}],"shares":53.0,"price":668.54,"changePercent":0.6291548883906813,"changeValue":13683.54,"id":"18641721","value":35432.62,"account":"6834413"},"18613735":{"shares":0.0,"symbol":"","cost":0.0,"price":null,"inception":"Oct 24, 2020","description":"Cash and Money Market","changeValue":4.76,"id":"18613735","type":15,"transactions":[],"value":4.76,"account":"6834413"}},"value":35437.380000000005},"segment":"multiple","showPopup":null,"changePercent":1.1177614250071062,"changeValue":61935.44000000001,"value":117345.69000000002,"6216793":{"isinactive":"false","cost":0.0,"inception":"Jan 3, 2016","description":"Wealthfront - My Roth IRA","isretirement":false,"isbrokerage":false,"sorted":{"performance-asc":["15771339","15771335","15771341"],"performance-dsc":["15771335","15771341","15771339"],"value":["15771341","15771335","15771339"],"movement":["15771339","15771335","15771341"]},"dormant":false,"changeValue":4.440892098500626E-16,"id":"6216793","investmentType":"Investment, Taxable","holdings":{"15771339":{"symbol":"","cost":-7.77,"inception":"Feb 3, 2018","description":"Cash","type":15,"transactions":[],"shares":0.0,"price":null,"changePercent":-0.0,"changeValue":0.0,"id":"15771339","value":-7.77,"account":"6216793"},"15771335":{"symbol":"","cost":2.65,"inception":"Feb 3, 2018","description":"Corporate Bonds","type":15,"transactions":[],"shares":0.0,"price":null,"changePercent":0.0,"changeValue":0.0,"id":"15771335","value":2.65,"account":"6216793"},"15771341":{"symbol":"","cost":5.12,"inception":"Feb 3, 2018","description":"Emerging Market Bonds","type":15,"transactions":[],"shares":0.0,"price":null,"changePercent":0.0,"changeValue":0.0,"id":"15771341","value":5.12,"account":"6216793"}},"value":4.440892098500626E-16},"lastDate":"Wed Jul 14 close"})

# load to dict
my_dict = json.loads(input) 

NameError: name 'false' is not defined

In [147]:

pp.pprint(bills)

[{'aggregationDueAmount': 0.0,
  'allowedPaymentMethodOptions': ['ACH', 'DEBIT_CARD', 'CREDIT_CARD'],
  'billDetailsList': [{'billDetailsType': 'UTILITY',
                       'name': 'Payments',
                       'value': 228.7}],
  'billStatus': 'FULL',
  'contentAccountRef': {'connectedAccounts': [{'contentAccountRef': '79fb33687147ae2cb28aa07af3739811',
                                               'contentLoginRef': 'c52b0209-04d5-3922-b6cb-8cbb2bae22f3',
                                               'contentProviderId': 2,
                                               'contentProviderName': 'Intuit '
                                                                      'Mint '
                                                                      'Bills'}],
                        'contentAccountRef': '79fb33687147ae2cb28aa07af3739811',
                        'contentLoginRef': 'c52b0209-04d5-3922-b6cb-8cbb2bae22f3',
                        'contentProviderId': 2,
     

In [148]:
print(test_df.head())


                                   contentAccountRef  \
0  {'contentProviderId': 2, 'contentProviderName'...   
1  {'contentProviderId': 2, 'contentProviderName'...   
2  {'contentProviderId': 2, 'contentProviderName'...   
3  {'contentProviderId': 2, 'contentProviderName'...   
4  {'contentProviderId': 2, 'contentProviderName'...   

                                         providerRef         model  \
0  {'providerId': 'da579b9a-a53a-4a0d-91e3-8cf9c7...  REGULAR_BILL   
1  {'providerId': '75f8d952-be81-4e22-951e-41b291...   CREDIT_CARD   
2  {'providerId': 'f141569e-9e1b-4ad5-a8c7-8cb3d5...     INSURANCE   
3  {'providerId': '5a386b71-bdf5-4c41-a1a5-3ad5d4...          LOAN   
4  {'providerId': '7e3c2c96-44da-4d23-b966-2d34f6...   CREDIT_CARD   

      dueDate  aggregationDueAmount  dueAmount currency  pastDueAmount  \
0  2018-03-20                  0.00       0.00      USD            0.0   
1  2021-07-21               1591.23    1591.23      USD            NaN   
2  2020-02-03       

In [149]:

print(test_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 29 columns):
contentAccountRef               6 non-null object
providerRef                     6 non-null object
model                           6 non-null object
dueDate                         6 non-null object
aggregationDueAmount            6 non-null float64
dueAmount                       6 non-null float64
currency                        6 non-null object
pastDueAmount                   2 non-null float64
statementAmount                 4 non-null float64
statementDate                   4 non-null object
billStatus                      6 non-null object
pastDueStatus                   6 non-null object
isPayable                       6 non-null bool
isHistory                       6 non-null bool
institutionType                 6 non-null object
institutionId                   6 non-null object
billDetailsList                 5 non-null object
isVisible                       6 non-null bool
a

In [150]:
flat_bills = nested_to_record(bills, sep='_')

In [151]:
flat_bills_df = pd.DataFrame(flat_bills)
flat_bills_df

,model,dueDate,aggregationDueAmount,dueAmount,currency,pastDueAmount,statementAmount,statementDate,billStatus,pastDueStatus,isPayable,isHistory,institutionType,institutionId,billDetailsList,isVisible,allowedPaymentMethodOptions,id,contentAccountRef_contentProviderId,contentAccountRef_contentProviderName,contentAccountRef_contentLoginRef,contentAccountRef_contentAccountRef,contentAccountRef_connectedAccounts,providerRef_providerId,providerRef_providerName,providerRef_contentProviderRef,metaData_createdDate,metaData_link,lastDigits,minimumPaymentDue,aggregationMinimumPaymentDue,lastPaymentDate,autoPaid,accountNumber,paymentResetDate,lastPaymentAmount
0,REGULAR_BILL,2018-03-20,0.00,0.00,USD,0.0,0.00,2018-02-28,FULL,FALSE,False,False,LINKED,8897093450193760206,"[{'billDetailsType': 'UTILITY', 'name': 'Payme...",True,"[ACH, DEBIT_CARD, CREDIT_CARD]",7326903310100285047,2,Intuit Mint Bills,c52b0209-04d5-3922-b6cb-8cbb2bae22f3,79fb33687147ae2cb28aa07af3739811,"[{'contentProviderId': 2, 'contentProviderName...",da579b9a-a53a-4a0d-91e3-8cf9c75fca28,xfinity,"[{'contentProviderId': 1, 'contentProviderName...",2017-01-13T20:44:20Z,[{'href': '/v2/payer/institutions/889709345019...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CREDIT_CARD,2021-07-21,1591.23,1591.23,USD,NaN,1591.23,2021-06-25,NOT_PAID,FALSE,False,False,LINKED,3034292192760981176,"[{'billDetailsType': 'CREDIT_CARD', 'name': 'B...",True,"[ACH, DEBIT_CARD]",4576616495530057800,2,Intuit Mint Bills,3034292192760981176,75391669130,"[{'contentProviderId': 1, 'contentProviderName...",75f8d952-be81-4e22-951e-41b291d211f2,American Express Corporate or Personal Credit ...,"[{'contentProviderId': 1, 'contentProviderName...",2019-08-03T14:23:49Z,[{'href': '/v2/payer/institutions/303429219276...,1003,40.0,40.0,2021-07-10,NaN,NaN,NaN,NaN
2,INSURANCE,2020-02-03,0.00,0.00,USD,0.0,NaN,NaN,FULL,FALSE,False,False,LINKED,1489353012757094401,NaN,True,"[ACH, DEBIT_CARD, CREDIT_CARD]",2930261357954217868,2,Intuit Mint Bills,dce6c3b2-aced-3977-9454-bb5f58eac7bb,39c2342139fcc5e34c526b87022f7773,"[{'contentProviderId': 2, 'contentProviderName...",f141569e-9e1b-4ad5-a8c7-8cb3d545703d,Progressive,"[{'contentProviderId': 1, 'contentProviderName...",2018-02-12T00:44:11Z,[{'href': '/v2/payer/institutions/148935301275...,6768,NaN,NaN,NaN,True,915556768,2020-01-29T05:09:48.844+00:00,NaN
3,LOAN,2021-08-06,554.16,554.16,USD,NaN,NaN,NaN,NOT_PAID,FALSE,False,False,LINKED,7253342564148930261,"[{'billDetailsType': 'LOAN', 'name': 'XXXXX072...",True,"[ACH, DEBIT_CARD]",1890193819581491325,2,Intuit Mint Bills,7253342564148930261,75370821235,"[{'contentProviderId': 1, 'contentProviderName...",5a386b71-bdf5-4c41-a1a5-3ad5d411cc8d,Honda Financial Services,"[{'contentProviderId': 1, 'contentProviderName...",2019-08-03T14:23:49Z,[{'href': '/v2/payer/institutions/725334256414...,0727,NaN,NaN,2021-07-01,True,9719854306462430727,NaN,554.16
4,CREDIT_CARD,2020-09-14,59.00,59.00,USD,NaN,59.00,2020-08-17,NOT_PAID,FALSE,False,False,LINKED,1497807465002559901,"[{'billDetailsType': 'CREDIT_CARD', 'name': 'V...",True,"[ACH, DEBIT_CARD]",4658404481527747228,2,Intuit Mint Bills,1497807465002559901,75180217336,"[{'contentProviderId': 1, 'contentProviderName...",7e3c2c96-44da-4d23-b966-2d34f65b3401,Capital One - Credit Cards,"[{'contentProviderId': 1, 'contentProviderName...",2019-08-03T14:23:47Z,[{'href': '/v2/payer/institutions/149780746500...,5671,NaN,NaN,2019-09-14,NaN,NaN,NaN,59.00
5,CREDIT_CARD,2021-08-02,353.85,353.85,USD,NaN,353.85,2021-07-05,NOT_PAID,FALSE,False,False,LINKED,5071473431000519851,"[{'billDetailsType': 'CREDIT_CARD', 'name': 'C...",True,"[ACH, DEBIT_CARD]",6060316510470412721,2,Intuit Mint Bills,5071473431000519851,75398097885,"[{'contentProviderId': 1, 'contentProviderName...",867c4a38-0d75-46ff-a4c2-f44cb63be2c9,Chase Bank,"[{'contentProviderId': 1, 'contentProviderName...",2019-08-03T14:23:48Z,[{'href': '/v2/payer/institutions/507147343100...,0995,40.0,40.0,2021-07-02,True,NaN,NaN,283.93


In [155]:
flat_bills_df['allowedPaymentMethods'] = flat_bills_df.allowedPaymentMethodOptions.apply(lambda x : "-".join(x))

In [156]:
flat_bills_df

,model,dueDate,aggregationDueAmount,dueAmount,currency,pastDueAmount,statementAmount,statementDate,billStatus,pastDueStatus,isPayable,isHistory,institutionType,institutionId,billDetailsList,isVisible,allowedPaymentMethodOptions,id,contentAccountRef_contentProviderId,contentAccountRef_contentProviderName,contentAccountRef_contentLoginRef,contentAccountRef_contentAccountRef,contentAccountRef_connectedAccounts,providerRef_providerId,providerRef_providerName,providerRef_contentProviderRef,metaData_createdDate,metaData_link,lastDigits,minimumPaymentDue,aggregationMinimumPaymentDue,lastPaymentDate,autoPaid,accountNumber,paymentResetDate,lastPaymentAmount,test_lambda,allowedPaymentMethods
0,REGULAR_BILL,2018-03-20,0.00,0.00,USD,0.0,0.00,2018-02-28,FULL,FALSE,False,False,LINKED,8897093450193760206,"[{'billDetailsType': 'UTILITY', 'name': 'Payme...",True,"[ACH, DEBIT_CARD, CREDIT_CARD]",7326903310100285047,2,Intuit Mint Bills,c52b0209-04d5-3922-b6cb-8cbb2bae22f3,79fb33687147ae2cb28aa07af3739811,"[{'contentProviderId': 2, 'contentProviderName...",da579b9a-a53a-4a0d-91e3-8cf9c75fca28,xfinity,"[{'contentProviderId': 1, 'contentProviderName...",2017-01-13T20:44:20Z,[{'href': '/v2/payer/institutions/889709345019...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ACH-DEBIT_CARD-CREDIT_CARD,ACH-DEBIT_CARD-CREDIT_CARD
1,CREDIT_CARD,2021-07-21,1591.23,1591.23,USD,NaN,1591.23,2021-06-25,NOT_PAID,FALSE,False,False,LINKED,3034292192760981176,"[{'billDetailsType': 'CREDIT_CARD', 'name': 'B...",True,"[ACH, DEBIT_CARD]",4576616495530057800,2,Intuit Mint Bills,3034292192760981176,75391669130,"[{'contentProviderId': 1, 'contentProviderName...",75f8d952-be81-4e22-951e-41b291d211f2,American Express Corporate or Personal Credit ...,"[{'contentProviderId': 1, 'contentProviderName...",2019-08-03T14:23:49Z,[{'href': '/v2/payer/institutions/303429219276...,1003,40.0,40.0,2021-07-10,NaN,NaN,NaN,NaN,ACH-DEBIT_CARD,ACH-DEBIT_CARD
2,INSURANCE,2020-02-03,0.00,0.00,USD,0.0,NaN,NaN,FULL,FALSE,False,False,LINKED,1489353012757094401,NaN,True,"[ACH, DEBIT_CARD, CREDIT_CARD]",2930261357954217868,2,Intuit Mint Bills,dce6c3b2-aced-3977-9454-bb5f58eac7bb,39c2342139fcc5e34c526b87022f7773,"[{'contentProviderId': 2, 'contentProviderName...",f141569e-9e1b-4ad5-a8c7-8cb3d545703d,Progressive,"[{'contentProviderId': 1, 'contentProviderName...",2018-02-12T00:44:11Z,[{'href': '/v2/payer/institutions/148935301275...,6768,NaN,NaN,NaN,True,915556768,2020-01-29T05:09:48.844+00:00,NaN,ACH-DEBIT_CARD-CREDIT_CARD,ACH-DEBIT_CARD-CREDIT_CARD
3,LOAN,2021-08-06,554.16,554.16,USD,NaN,NaN,NaN,NOT_PAID,FALSE,False,False,LINKED,7253342564148930261,"[{'billDetailsType': 'LOAN', 'name': 'XXXXX072...",True,"[ACH, DEBIT_CARD]",1890193819581491325,2,Intuit Mint Bills,7253342564148930261,75370821235,"[{'contentProviderId': 1, 'contentProviderName...",5a386b71-bdf5-4c41-a1a5-3ad5d411cc8d,Honda Financial Services,"[{'contentProviderId': 1, 'contentProviderName...",2019-08-03T14:23:49Z,[{'href': '/v2/payer/institutions/725334256414...,0727,NaN,NaN,2021-07-01,True,9719854306462430727,NaN,554.16,ACH-DEBIT_CARD,ACH-DEBIT_CARD
4,CREDIT_CARD,2020-09-14,59.00,59.00,USD,NaN,59.00,2020-08-17,NOT_PAID,FALSE,False,False,LINKED,1497807465002559901,"[{'billDetailsType': 'CREDIT_CARD', 'name': 'V...",True,"[ACH, DEBIT_CARD]",4658404481527747228,2,Intuit Mint Bills,1497807465002559901,75180217336,"[{'contentProviderId': 1, 'contentProviderName...",7e3c2c96-44da-4d23-b966-2d34f65b3401,Capital One - Credit Cards,"[{'contentProviderId': 1, 'contentProviderName...",2019-08-03T14:23:47Z,[{'href': '/v2/payer/institutions/149780746500...,5671,NaN,NaN,2019-09-14,NaN,NaN,NaN,59.00,ACH-DEBIT_CARD,ACH-DEBIT_CARD
5,CREDIT_CARD,2021-08-02,353.85,353.85,USD,NaN,353.85,2021-07-05,NOT_PAID,FALSE,False,False,LINKED,5071473431000519851,"[{'billDetailsType': 'CREDIT_CARD', 'name': 'C...",True,"[ACH, DEBIT_CARD]",6060316510470412721,2,Intuit Mint Bills,5071473431000519851,75398097885,"[{'contentProviderId': 1, 'contentProviderName...",867c4a38-0d75-46ff-

In [355]:
flat_bills_df.metaData_link[2]

[{'href': '/v2/payer/institutions/1489353012757094401', 'rel': 'institution'}]

In [159]:
flat_bills_df.billDetailsList[1]

[{'billDetailsType': 'CREDIT_CARD',
  'name': 'Bonvoy Amex Card',
  'number': '1003',
  'availableBalanceAmount': 2305.6,
  'availableBalanceType': 'DEBIT',
  'creditLimit': 10000.0,
  'status': 'OTHER'}]

In [160]:
flat_bills_df.billDetailsList[2]

nan

In [194]:
flat_bills_df.billDetailsList[4][0]['billDetailsType']

'CREDIT_CARD'

In [243]:
def goByDFRow(df):
    df['billType'] = ""
    
    for index, row in df.iterrows():
#         print(row.billDetailsList)
    
        #Get the billtype
        try:
            
            df.billType.loc[index] = row.billDetailsList[0]['billDetailsType']
        except:
            df.billType.loc[index] = "Bill Type Not Found"
        
        if 
        
#         print(row)
    print(df)
    
    
    
    
    
    
    return df

#Working Version

In [342]:
def goByDFRowGetBill(df):
    
    
    # Create columns to hold data
    df['billDetails_Type'] = None
    
    # Credit Card Info
    df['billDetails_creditCardNameAndEndingNumber'] = None
    df['billDetails_creditCardAvailableBalanceAmount'] = None
    df['billDetails_creditCardAvailableBalanceType'] = None
    df['billDetails_creditCardCreditLimit'] = None
    df['billDetails_creditCardPurchasesApr'] = None
    df['billDetails_creditCardStatus'] = None

    
    # Utility Payments
    df['billDetails_utilityName']  = None
    df['billDetails_utilityValue'] = None

    # Loan Details
    df['billDetails_loanNumber'] = None
    df['billDetails_loanCurrentBalance'] = None
    df['billDetails_loanPayOffAmount'] = None
    df['billDetails_loanEndDate'] = None
    df['billDetails_loanRemaningPayments'] = None
    df['billDetails_loanStatus'] = None

    
    NF = "Not Found"
    
    
    for index , _ in df.iterrows():
#         print(row.billDetailsList)
        try:
                        df.loc[index, 'billDetails_Type'] = df.loc[index, 'billDetailsList'][0]['billDetailsType']
        except:
            df.loc[index, 'billDetails_Type'] = "Bill Type Not Found"
        
        if  df.loc[index, 'billDetails_Type'] == 'CREDIT_CARD':
            try:
                
                df.loc[index, 'billDetails_creditCardNameAndEndingNumber'] =  df.loc[index, 'billDetailsList'][0]['name'] + " - " + df.loc[index, 'billDetailsList'][0]['number']
            except:
                df.loc[index, 'billDetails_creditCardNameAndEndingNumber'] = NF
            
            try:
                df.loc[index, 'billDetails_creditCardAvailableBalanceAmount'] =  df.loc[index, 'billDetailsList'][0]['availableBalanceAmount']
            except:
                df.loc[index, 'billDetails_creditCardAvailableBalanceAmount'] = NF
            try:
                df.loc[index, 'billDetails_creditCardAvailableBalanceType'] =  df.loc[index, 'billDetailsList'][0]['availableBalanceType']
            except:
                df.loc[index, 'billDetails_creditCardAvailableBalanceType'] = NF
            try:
                df.loc[index, 'billDetails_creditCardCreditLimit'] =  df.loc[index, 'billDetailsList'][0]['creditLimit']
            except:
                df.loc[index, 'billDetails_creditCardCreditLimit'] = NF
            try:
                df.loc[index, 'billDetails_creditCardPurchasesApr'] =  df.loc[index, 'billDetailsList'][0]['purchasesApr']
            except:
                df.loc[index, 'billDetails_creditCardPurchasesApr'] = NF
            try:
                df.loc[index, 'billDetails_creditCardStatus'] =  df.loc[index, 'billDetailsList'][0]['status']
            except:
                df.loc[index, 'billDetails_creditCardStatus'] = NF
                
        elif  df.loc[index, 'billDetails_Type'] == 'UTILITY':
            
            try:
                
                df.loc[index,'billDetails_utilityName']  = df.loc[index, 'billDetailsList'][0]['name']
            
            except:
                df.loc[index, 'billDetails_utilityName'] = NF
            
            try:
                
                df.loc[index, 'billDetails_utilityValue'] = df.loc[index, 'billDetailsList'][0]['value']
            
            except:
                df[index, 'billDetails_utilityValue'] = NF

                
                
        elif df.loc[index, 'billDetails_Type'] == 'LOAN':
            
            try:
                df.loc[index, 'billDetails_loanNumber'] = df.loc[index, 'billDetailsList'][0]['name']
            
            except:
                df.loc[index, 'billDetails_loanNumber'] = NF
                
            try:   
                df.loc[index, 'billDetails_loanCurrentBalance'] = df.loc[index, 'billDetailsList'][0]['currentBalance']
            
            except:
                df.loc[index, 'billDetails_loanCurrentBalance'] = NF
                
            try:
                df.loc[index, 'billDetails_loanPayOffAmount'] = df.loc[index, 'billDetailsList'][0]['payoffAmount']
            
            except: 
                
                df.loc[index, 'billDetails_loanPayOffAmount'] = NF
                
            try:
                df.loc[index, 'billDetails_loanEndDate'] =  df.loc[index, 'billDetailsList'][0]['loanEndDate']
            
            except:
                df.loc[index, 'billDetails_loanEndDate'] = NF
            
            try:
                df.loc[index, 'billDetails_loanRemaningPayments'] = df.loc[index, 'billDetailsList'][0]['remainingPayments']
            
            except:
                df.loc[index, 'billDetails_loanRemaningPayments'] = NF

            try:
                df.loc[index, 'billDetails_loanStatus'] =  df.loc[index, 'billDetailsList'][0]['loanStatus']
            except:
                df.loc[index, 'billDetails_loanStatus'] = NF

        
        
#         print(row)
    print(df)
    return df

# {'billDetailsType': 'UTILITY', 'name': 'Payments', 'value': 228.7}




# [{'billDetailsType': 'LOAN',
#   'name': 'XXXXX0727',
#   'currentBalance': 14683.41,
#   'payoffAmount': 14683.41,
#   'loanEndDate': '2023-11-06T16:00:00.000+00:00',
#   'remainingPayments': 28.0,
#   'loanStatus': 'OTHER'}]



In [343]:
test_df = goByDFRowGetBill(flat_bills_df[['billDetailsList']])

                                     billDetailsList     billDetails_Type  \
0  [{'billDetailsType': 'UTILITY', 'name': 'Payme...              UTILITY   
1  [{'billDetailsType': 'CREDIT_CARD', 'name': 'B...          CREDIT_CARD   
2                                                NaN  Bill Type Not Found   
3  [{'billDetailsType': 'LOAN', 'name': 'XXXXX072...                 LOAN   
4  [{'billDetailsType': 'CREDIT_CARD', 'name': 'V...          CREDIT_CARD   
5  [{'billDetailsType': 'CREDIT_CARD', 'name': 'C...          CREDIT_CARD   

  billDetails_creditCardNameAndEndingNumber  \
0                                      None   
1                   Bonvoy Amex Card - 1003   
2                                      None   
3                                      None   
4                            Venture - 5671   
5                        CREDIT CARD - 0995   

  billDetails_creditCardAvailableBalanceAmount  \
0                                         None   
1                             

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

In [344]:
test_df

,billDetailsList,billDetails_Type,billDetails_creditCardNameAndEndingNumber,billDetails_creditCardAvailableBalanceAmount,billDetails_creditCardAvailableBalanceType,billDetails_creditCardCreditLimit,billDetails_creditCardPurchasesApr,billDetails_creditCardStatus,billDetails_utilityName,billDetails_utilityValue,billDetails_loanNumber,billDetails_loanCurrentBalance,billDetails_loanPayOffAmount,billDetails_loanEndDate,billDetails_loanRemaningPayments,billDetails_loanStatus
0,"[{'billDetailsType': 'UTILITY', 'name': 'Payme...",UTILITY,None,None,None,None,None,None,Payments,228.7,None,None,None,None,None,None
1,"[{'billDetailsType': 'CREDIT_CARD', 'name': 'B...",CREDIT_CARD,Bonvoy Amex Card - 1003,2305.6,DEBIT,10000,Not Found,OTHER,None,None,None,None,None,None,None,None
2,NaN,Bill Type Not Found,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,"[{'billDetailsType': 'LOAN', 'name': 'XXXXX072...",LOAN,None,None,None,None,None,None,None,None,XXXXX0727,14683.4,14683.4,2023-11-06T16:00:00.000+00:00,28,OTHER
4,"[{'billDetailsType': 'CREDIT_CARD', 'name': 'V...",CREDIT_CARD,Venture - 5671,59,DEBIT,Not Found,Not Found,OTHER,None,None,None,None,None,None,None,None
5,"[{'billDetailsType': 'CREDIT_CARD', 'name': 'C...",CREDIT_CARD,CREDIT CARD - 0995,353.85,DEBIT,13000,22.99,OTHER,None,None,None,None,None,None,None,None


In [341]:
flat_bills_df.iloc[1]['billDetailsList'][0]

{'billDetailsType': 'CREDIT_CARD',
 'name': 'Bonvoy Amex Card',
 'number': '1003',
 'availableBalanceAmount': 2305.6,
 'availableBalanceType': 'DEBIT',
 'creditLimit': 10000.0,
 'status': 'OTHER'}

In [356]:
flat_bills_df

,model,dueDate,aggregationDueAmount,dueAmount,currency,pastDueAmount,statementAmount,statementDate,billStatus,pastDueStatus,isPayable,isHistory,institutionType,institutionId,billDetailsList,isVisible,allowedPaymentMethodOptions,id,contentAccountRef_contentProviderId,contentAccountRef_contentProviderName,contentAccountRef_contentLoginRef,contentAccountRef_contentAccountRef,contentAccountRef_connectedAccounts,providerRef_providerId,providerRef_providerName,providerRef_contentProviderRef,metaData_createdDate,metaData_link,lastDigits,minimumPaymentDue,aggregationMinimumPaymentDue,lastPaymentDate,autoPaid,accountNumber,paymentResetDate,lastPaymentAmount,test_lambda,allowedPaymentMethods
0,REGULAR_BILL,2018-03-20,0.00,0.00,USD,0.0,0.00,2018-02-28,FULL,FALSE,False,False,LINKED,8897093450193760206,"[{'billDetailsType': 'UTILITY', 'name': 'Payme...",True,"[ACH, DEBIT_CARD, CREDIT_CARD]",7326903310100285047,2,Intuit Mint Bills,c52b0209-04d5-3922-b6cb-8cbb2bae22f3,79fb33687147ae2cb28aa07af3739811,"[{'contentProviderId': 2, 'contentProviderName...",da579b9a-a53a-4a0d-91e3-8cf9c75fca28,xfinity,"[{'contentProviderId': 1, 'contentProviderName...",2017-01-13T20:44:20Z,[{'href': '/v2/payer/institutions/889709345019...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ACH-DEBIT_CARD-CREDIT_CARD,ACH-DEBIT_CARD-CREDIT_CARD
1,CREDIT_CARD,2021-07-21,1591.23,1591.23,USD,NaN,1591.23,2021-06-25,NOT_PAID,FALSE,False,False,LINKED,3034292192760981176,"[{'billDetailsType': 'CREDIT_CARD', 'name': 'B...",True,"[ACH, DEBIT_CARD]",4576616495530057800,2,Intuit Mint Bills,3034292192760981176,75391669130,"[{'contentProviderId': 1, 'contentProviderName...",75f8d952-be81-4e22-951e-41b291d211f2,American Express Corporate or Personal Credit ...,"[{'contentProviderId': 1, 'contentProviderName...",2019-08-03T14:23:49Z,[{'href': '/v2/payer/institutions/303429219276...,1003,40.0,40.0,2021-07-10,NaN,NaN,NaN,NaN,ACH-DEBIT_CARD,ACH-DEBIT_CARD
2,INSURANCE,2020-02-03,0.00,0.00,USD,0.0,NaN,NaN,FULL,FALSE,False,False,LINKED,1489353012757094401,NaN,True,"[ACH, DEBIT_CARD, CREDIT_CARD]",2930261357954217868,2,Intuit Mint Bills,dce6c3b2-aced-3977-9454-bb5f58eac7bb,39c2342139fcc5e34c526b87022f7773,"[{'contentProviderId': 2, 'contentProviderName...",f141569e-9e1b-4ad5-a8c7-8cb3d545703d,Progressive,"[{'contentProviderId': 1, 'contentProviderName...",2018-02-12T00:44:11Z,[{'href': '/v2/payer/institutions/148935301275...,6768,NaN,NaN,NaN,True,915556768,2020-01-29T05:09:48.844+00:00,NaN,ACH-DEBIT_CARD-CREDIT_CARD,ACH-DEBIT_CARD-CREDIT_CARD
3,LOAN,2021-08-06,554.16,554.16,USD,NaN,NaN,NaN,NOT_PAID,FALSE,False,False,LINKED,7253342564148930261,"[{'billDetailsType': 'LOAN', 'name': 'XXXXX072...",True,"[ACH, DEBIT_CARD]",1890193819581491325,2,Intuit Mint Bills,7253342564148930261,75370821235,"[{'contentProviderId': 1, 'contentProviderName...",5a386b71-bdf5-4c41-a1a5-3ad5d411cc8d,Honda Financial Services,"[{'contentProviderId': 1, 'contentProviderName...",2019-08-03T14:23:49Z,[{'href': '/v2/payer/institutions/725334256414...,0727,NaN,NaN,2021-07-01,True,9719854306462430727,NaN,554.16,ACH-DEBIT_CARD,ACH-DEBIT_CARD
4,CREDIT_CARD,2020-09-14,59.00,59.00,USD,NaN,59.00,2020-08-17,NOT_PAID,FALSE,False,False,LINKED,1497807465002559901,"[{'billDetailsType': 'CREDIT_CARD', 'name': 'V...",True,"[ACH, DEBIT_CARD]",4658404481527747228,2,Intuit Mint Bills,1497807465002559901,75180217336,"[{'contentProviderId': 1, 'contentProviderName...",7e3c2c96-44da-4d23-b966-2d34f65b3401,Capital One - Credit Cards,"[{'contentProviderId': 1, 'contentProviderName...",2019-08-03T14:23:47Z,[{'href': '/v2/payer/institutions/149780746500...,5671,NaN,NaN,2019-09-14,NaN,NaN,NaN,59.00,ACH-DEBIT_CARD,ACH-DEBIT_CARD
5,CREDIT_CARD,2021-08-02,353.85,353.85,USD,NaN,353.85,2021-07-05,NOT_PAID,FALSE,False,False,LINKED,5071473431000519851,"[{'billDetailsType': 'CREDIT_CARD', 'name': 'C...",True,"[ACH, DEBIT_CARD]",6060316510470412721,2,Intuit Mint Bills,5071473431000519851,75398097885,"[{'contentProviderId': 1, 'contentProviderName...",867c4a38-0d75-46ff-

In [317]:
flat_bills_df.billDetailsList[0]

[{'billDetailsType': 'UTILITY', 'name': 'Payments', 'value': 228.7}]

In [338]:
flat_bills_df.billDetailsList[3]

[{'billDetailsType': 'LOAN',
  'name': 'XXXXX0727',
  'currentBalance': 14683.41,
  'payoffAmount': 14683.41,
  'loanEndDate': '2023-11-06T16:00:00.000+00:00',
  'remainingPayments': 28.0,
  'loanStatus': 'OTHER'}]

In [143]:
flat_bills_df['test_lambda'] = flat_bills_df.allowedPaymentMethodOptions.apply(lambda x : "-".join(x))

In [140]:
flat_bills_df['test_apply'] = flat_bills_df.apply(getPaymentMethods_df(flat_bills_df), axis=1)

0    [ACH, DEBIT_CARD, CREDIT_CARD]
1                 [ACH, DEBIT_CARD]
2    [ACH, DEBIT_CARD, CREDIT_CARD]
3                 [ACH, DEBIT_CARD]
4                 [ACH, DEBIT_CARD]
5                 [ACH, DEBIT_CARD]
Name: allowedPaymentMethodOptions, dtype: object
<class 'pandas.core.series.Series'>
0    ACH-DEBIT_CARD
1    ACH-DEBIT_CARD
2    ACH-DEBIT_CARD
3    ACH-DEBIT_CARD
4    ACH-DEBIT_CARD
5    ACH-DEBIT_CARD
Name: test_col, dtype: object


AttributeError: 'ACH-DEBIT_CARD' is not a valid function for 'Series' object

In [144]:
flat_bills_df['test_lambda']

0    ACH-DEBIT_CARD-CREDIT_CARD
1                ACH-DEBIT_CARD
2    ACH-DEBIT_CARD-CREDIT_CARD
3                ACH-DEBIT_CARD
4                ACH-DEBIT_CARD
5                ACH-DEBIT_CARD
Name: test_lambda, dtype: object

In [74]:
test_payment = ['ACH', 'DEBIT_CARD', 'CREDIT_CARD']

In [76]:
print(getPaymentMethods(test_payment))

ACH-DEBIT_CARD-CREDIT_CARD


In [51]:
flat_bills_df.loc[['institutionId','billDetailsList']]

KeyError: "None of [Index(['institutionId', 'billDetailsList'], dtype='object')] are in the [index]"

In [29]:
flat_bills_2 = nested_to_record(flat_bills, sep="_")
flat_bills_2_df = pd.DataFrame(flat_bills_2)
print(flat_bills_2_df)

          model     dueDate  aggregationDueAmount  dueAmount currency  \
0  REGULAR_BILL  2018-03-20                  0.00       0.00      USD   
1   CREDIT_CARD  2021-07-21               1591.23    1591.23      USD   
2     INSURANCE  2020-02-03                  0.00       0.00      USD   
3          LOAN  2021-08-06                554.16     554.16      USD   
4   CREDIT_CARD  2020-09-14                 59.00      59.00      USD   
5   CREDIT_CARD  2021-08-02                353.85     353.85      USD   

   pastDueAmount  statementAmount statementDate billStatus pastDueStatus  ...  \
0            0.0             0.00    2018-02-28       FULL         FALSE  ...   
1            NaN          1591.23    2021-06-25   NOT_PAID         FALSE  ...   
2            0.0              NaN           NaN       FULL         FALSE  ...   
3            NaN              NaN           NaN   NOT_PAID         FALSE  ...   
4            NaN            59.00    2020-08-17   NOT_PAID         FALSE  ...   
5 

In [20]:
flat_bills_2[0]

{'model': 'REGULAR_BILL',
 'dueDate': '2018-03-20',
 'aggregationDueAmount': 0.0,
 'dueAmount': 0.0,
 'currency': 'USD',
 'pastDueAmount': 0.0,
 'statementAmount': 0.0,
 'statementDate': '2018-02-28',
 'billStatus': 'FULL',
 'pastDueStatus': 'FALSE',
 'isPayable': False,
 'isHistory': False,
 'institutionType': 'LINKED',
 'institutionId': '8897093450193760206',
 'billDetailsList': [{'billDetailsType': 'UTILITY',
   'name': 'Payments',
   'value': 228.7}],
 'isVisible': True,
 'allowedPaymentMethodOptions': ['ACH', 'DEBIT_CARD', 'CREDIT_CARD'],
 'id': '7326903310100285047',
 'contentAccountRef_contentProviderId': 2,
 'contentAccountRef_contentProviderName': 'Intuit Mint Bills',
 'contentAccountRef_contentLoginRef': 'c52b0209-04d5-3922-b6cb-8cbb2bae22f3',
 'contentAccountRef_contentAccountRef': '79fb33687147ae2cb28aa07af3739811',
 'contentAccountRef_connectedAccounts': [{'contentProviderId': 2,
   'contentProviderName': 'Intuit Mint Bills',
   'contentLoginRef': 'c52b0209-04d5-3922-b6cb-

In [28]:
bills['billdingDetailsList'][0]

TypeError: list indices must be integers or slices, not str